# Pioneering experiments for keyless joining of tables

In this notebook we explore methods for stitching dataframes together without any shared variables or keys. For example, imagine that a dataframe "A" contains all categorical variables and another dataframe "B" contains all numerical variables. We would like to join these two tables together to obtain plausible results.

The experiments in this notebook explore if we can learn rules or methods for joining tables together without any shared keys.

In [1]:
### Prepare the datasets for the experiments

import pandas as pd

df = pd.read_csv('datasets/diabetes.csv')

df_train = df.sample(frac=0.5, random_state=42) # use half of the samples as references to learn how to do joining
df_train.to_csv('datasets/diabetes_train.csv', index=False)

df_test = df.drop(df_train.index)
df_test.to_csv('datasets/diabetes_test.csv', index=False) # save test set for evaluation

df_frame_A = df_test[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness']]

df_frame_B = df_test[['Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']]

df_frame_A.to_csv('datasets/diabetes_test_A.csv', index=False)
df_frame_B.to_csv('datasets/diabetes_test_B.csv', index=False)

In [2]:
from syntheval.metrics.utility.metric_accuracy_difference import ClassificationAccuracy

def evaluate_joining(df_attempt):
    """Function to measure the fraction of items in df_attempt that are also in df_true"""
    df_true = pd.read_csv('datasets/diabetes_test.csv')

    num_true = df_attempt.isin(df_true).all(axis=1).sum()

    frac_true = num_true / len(df_test)

    metric = ClassificationAccuracy(df_true, df_attempt, analysis_target='Outcome', verbose=False)
    res = metric.evaluate()

    return pd.Series({"num_joins": len(df_attempt), "num_true": num_true, "frac_of_test": frac_true.round(5), 'avg_acc_diff': res['avg diff'].round(5), 'avg_acc_err': res['avg diff err'].round(5)})

In [3]:
### Baseline 1: Random join

df_A = pd.read_csv('datasets/diabetes_test_A.csv')
df_B = pd.read_csv('datasets/diabetes_test_B.csv')

df_A = df_A.sample(frac=1, random_state=42)
df_B = df_B.sample(frac=1, random_state=42)

df_attempt = pd.concat([df_A, df_B], axis=1).reset_index(drop=True)

evaluate_joining(df_attempt)

SynthEval: inferred categorical columns...


num_joins       384.00000
num_true          1.00000
frac_of_test      0.00260
avg_acc_diff      0.09899
avg_acc_err       0.01521
dtype: float64

In [4]:
### Implement match validator (simple classification model to predict if a join looks plausible)

from sklearn.neural_network import MLPClassifier

class classifier_validator:
    def __init__(self, df_train_A, df_train_B):
        df_join_train_true = pd.concat([df_train_A, df_train_B], axis=1).reset_index(drop=True)
        df_join_train_false = pd.concat([df_train_A.sample(frac=2, random_state=42, replace=True), df_train_B.sample(frac=2, random_state=42, replace=True)], axis=1).reset_index(drop=True)

        valid_joins_col = [1]*len(df_join_train_true)+[0]*len(df_join_train_false)

        df_join_train = pd.concat([df_join_train_true, df_join_train_false], axis=0).reset_index(drop=True)

        self.clf = MLPClassifier(random_state=42, max_iter=1000).fit(df_join_train, valid_joins_col)

    def validate(self, df_attempt):
        self.clf.predict(df_attempt)
        return df_attempt.loc[self.clf.predict(df_attempt)==1]

In [5]:
### Experiment 1: Naievely join take the cross product of the two datasets and route it throug hthe validator

df_train_A = pd.read_csv('datasets/diabetes_train.csv')[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness']]
df_train_B = pd.read_csv('datasets/diabetes_train.csv')[['Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']]

validator = classifier_validator(df_train_A, df_train_B)

df_A = pd.read_csv('datasets/diabetes_test_A.csv')
df_B = pd.read_csv('datasets/diabetes_test_B.csv')

df_attempt = df_A.merge(df_B, how='cross')

df_attempt_good_joins = validator.validate(df_attempt)

evaluate_joining(df_attempt_good_joins)

SynthEval: inferred categorical columns...


num_joins       22586.00000
num_true            0.00000
frac_of_test        0.00000
avg_acc_diff        0.06383
avg_acc_err         0.01957
dtype: float64

In [10]:
### Experiment 2: Do rounds of matching and elimination for a few steps removing items that are sucessfully matched

import numpy as np

df_train_A = pd.read_csv('datasets/diabetes_train.csv')[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness']]
df_train_B = pd.read_csv('datasets/diabetes_train.csv')[['Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']]

validator = classifier_validator(df_train_A, df_train_B)

df_A = pd.read_csv('datasets/diabetes_test_A.csv')
df_B = pd.read_csv('datasets/diabetes_test_B.csv')

def iterative_joining(df_A, df_B, validator, num_rounds):
    A_idx = list(df_A.index)
    B_idx = list(df_B.index)

    df_good_joins = None
    for i in range(num_rounds):
        np.random.shuffle(A_idx), np.random.shuffle(B_idx)

        df_attempt = pd.concat([df_A.iloc[A_idx].reset_index(drop=True), df_B.iloc[B_idx].reset_index(drop=True)], axis=1).reset_index(drop=True)

        df_attempt_good_joins = validator.validate(df_attempt)

        df_attempt_good_joins_idx = list(sorted(df_attempt_good_joins.index))

        [B_idx.pop(i) for i in df_attempt_good_joins_idx[::-1]]
        [A_idx.pop(i) for i in df_attempt_good_joins_idx[::-1]]
        
        if df_good_joins is None:
            df_good_joins = df_attempt_good_joins.reset_index(drop=True)
        else:
            df_good_joins = pd.concat([df_good_joins, df_attempt_good_joins], axis=0).reset_index(drop=True)
    return df_good_joins

df_good_joins = iterative_joining(df_A, df_B, validator ,100)

evaluate_joining(df_good_joins)

SynthEval: inferred categorical columns...


num_joins       219.00000
num_true          0.00000
frac_of_test      0.00000
avg_acc_diff      0.03443
avg_acc_err       0.01801
dtype: float64

We remark that using the validator on the cross product of matches improved performance by 1/3, but doing the iterative matching improved performance by 2/3ds. 

## Synthetic data generation

In the following, we will create synthetic data using two different methods, then use the joining validator to check if the joining is successful.

Finally we use SynthEval to evaluate the quality of the synthetic data compared with the "training set".

In [12]:
import time
from syntheval import SynthEval
from synthcity.plugins import Plugins

df_A = pd.read_csv('datasets/diabetes_test_A.csv')
df_B = pd.read_csv('datasets/diabetes_test_B.csv')

df_baseline = pd.concat([df_A, df_B], axis=1).reset_index(drop=True)

### Baseline
start = time.time()
model = Plugins().get('bayesian_network')
model.fit(df_baseline)
df_syn_baseline = model.generate(count=len(df_baseline)).dataframe()
print("Baseline time:", time.time()-start)

### Privacy Baseline
start = time.time()
model = Plugins().get('privbayes', epsilon=0.1, K=2)
model.fit(df_baseline)
df_syn_baseline_priv = model.generate(count=len(df_baseline)).dataframe()
print("Privacy Baseline time:", time.time()-start)

### Dataframe A:
start = time.time()
model = Plugins().get('bayesian_network')
model.fit(df_A)
df_syn_A = model.generate(count=3*len(df_A)).dataframe()
print("Data A time:", time.time()-start)

### Dataframe B:
start = time.time()
model = Plugins().get('bayesian_network')
model.fit(df_B)
df_syn_B = model.generate(count=3*len(df_B)).dataframe()
print("Data B time:", time.time()-start)

### Joining
df_valid = pd.read_csv('datasets/diabetes_test.csv')
validator = classifier_validator(df_A, df_B)

df_syn_joined = iterative_joining(df_syn_A, df_syn_B, validator, 100)
print(df_syn_joined.shape)

### Evaluate
df_test = pd.read_csv('datasets/diabetes_train.csv')
SE = SynthEval(df_baseline, df_test)
res, res_rank = SE.benchmark({"baseline:": df_syn_baseline,"privacy_base": df_syn_baseline_priv,"joined": df_syn_joined},analysis_target_var="Outcome",presets_file="full_eval")

res

[2024-11-14T09:28:02.918403+0100][33008][CRITICAL] module disabled: C:\Users\lautrup\AppData\Roaming\Python\Python310\site-packages\synthcity\plugins\generic\plugin_goggle.py
[2024-11-14T09:28:04.129756+0100][33008][CRITICAL] module disabled: C:\Users\lautrup\AppData\Roaming\Python\Python310\site-packages\synthcity\plugins\generic\plugin_goggle.py


Baseline time: 1.2123775482177734


100%|██████████| 8/8 [00:01<00:00,  6.26it/s]


  0%|          | 0/9 [00:00<?, ?it/s]

[2024-11-14T09:28:06.010495+0100][33008][CRITICAL] module disabled: C:\Users\lautrup\AppData\Roaming\Python\Python310\site-packages\synthcity\plugins\generic\plugin_goggle.py


Privacy Baseline time: 1.8807384967803955


[2024-11-14T09:28:06.606549+0100][33008][CRITICAL] module disabled: C:\Users\lautrup\AppData\Roaming\Python\Python310\site-packages\synthcity\plugins\generic\plugin_goggle.py


Data A time: 0.5950520038604736
Data B time: 1.486485481262207
(766, 9)
SynthEval: inferred categorical columns...


avg_dwm_diff           pca_eigval_diff       pca_eigvec_ang  \
                    value     error           value error          value   
dataset                                                                    
baseline:        0.007346  0.003911        0.017606   NaN       0.016558   
privacy_base     0.083269  0.004363        0.020669   NaN       0.077782   
joined           0.020911  0.003466        0.002429   NaN       0.053744   

                     avg_cio           corr_mat_diff        ... hit_rate  \
             error     value     error         value error  ...    error   
dataset                                                     ...            
baseline:      NaN  0.567196  0.129721      0.447003   NaN  ...      NaN   
privacy_base   NaN  0.131153   0.09566      1.686696   NaN  ...      NaN   
joined         NaN   0.04341   0.04341      1.866801   NaN  ...      NaN   

             eps_identif_risk       priv_loss_eps       mia_cls_risk  \
                        value error         value error        value   
dataset                                                                
baseline:            0.638021   NaN      0.234375   NaN     0.520861   
privacy_base         0.135417   NaN     -0.028646   NaN     0.457581   
joined               0.338542   NaN      0.039062   NaN     0.521878   

                             rank     u_rank    p_rank  
                 error                                  
dataset                                                 
baseline:     0.004302  13.183741  13.167913  0.015828  
privacy_base   0.01132   9.198375   1.294371  7.904003  
joined        0.011491  12.912025   6.575943  6.336081  

[3 rows x 49 columns]